In [86]:
import pandas as pd

# Define the file path
file_path = "C:\\Users\\e2023898\\Downloads\\test\\_CAU57PMSBXRG000R00_20240227_103941.txt"

# Import the txt file as a dataframe
df = pd.read_csv(file_path, delimiter="\t", header=None)
df = df.drop(17)

# Split the dataframe
test_id = df.iloc[:17]
test_id = test_id.loc[:,test_id.notna().any(axis=0)]
index_row = df.iloc[17]
remaining_df = df.iloc[17:]

# Initialize variables
datasets = {}
current_dataset = []
dataset_counter = 1
note_counter = 0

# Process the remaining rows
for index, row in remaining_df.iterrows():
    if row[0] == "Note":
        note_counter += 1
        if note_counter > 1:
            datasets[f"dataset_{dataset_counter}"] = pd.DataFrame(current_dataset)
            dataset_counter += 1
            current_dataset = []
    current_dataset.append(row)

# Append the last dataset if exists
if current_dataset:
    datasets[f"dataset_{dataset_counter}"] = pd.DataFrame(current_dataset)

# Create a dataframe for each dataset and show the names of the dataframes created
dataframe_names = []
for key, dataset in datasets.items():
    globals()[key] = dataset
    dataframe_names.append(key)

# Apply the operations to each dataset
for key, dataset in datasets.items():
    # Convert the second column to datetime, coercing errors
    dataset.iloc[3:, 1] = pd.to_datetime(dataset.iloc[3:, 1], format='%d/%m/%Y %H:%M:%S', errors='coerce')
    
    # Replace all commas with periods in columns 2 to the end
    dataset.iloc[:, 1:] = dataset.iloc[:, 1:].replace(',', '.', regex=True)
    
    # Convert subsequent columns to float, coercing errors
    # Convert the data from row 3 to the end of the dataframe to numeric
    for col in dataset.columns[2:]:
        dataset.iloc[2:, dataset.columns.get_loc(col)] = pd.to_numeric(dataset.iloc[2:, dataset.columns.get_loc(col)], errors='coerce')
    
    # Drop the first column
    #dataset.drop(dataset.columns[0], axis=1, inplace=True)
    
    # Update the dataset in the dictionary
    datasets[key] = dataset

    # Create the dataframe for store general data

riassunto = pd.DataFrame(columns=index_row)
new_columns = []
# Concatenate the new columns to the dataframe
riassunto = pd.concat([riassunto, pd.DataFrame(columns=new_columns)], axis=1)

# Assign values from datasets to riassunto sequentially in column 1
row_counter = 0
for key, dataset in datasets.items():
    value_to_assign = dataset.iloc[1, 0]  # Value from column 1 row 2
    riassunto.at[row_counter, 'Note'] = value_to_assign
    row_counter += 1

# Iterate over each dataset
for key, dataset in datasets.items():
    # Calculate the average values for each column starting from the third column
    averages = dataset.iloc[2:, 2:].mean()
    
    # Assign the average values to the corresponding row in riassunto
    row_index = dataframe_names.index(key)
    for col_index, avg_value in enumerate(averages, start=2):
        riassunto.iloc[row_index, col_index] = avg_value

# Create new columns for minimum and maximum values
avg_columns = []
min_columns = []
max_columns = []
for col in riassunto.columns[1:]:
    avg_columns.append(f"{col}_std")
    min_columns.append(f"{col}_min")
    max_columns.append(f"{col}_max")

# Append new columns to riassunto dataframe
riassunto = pd.concat([riassunto, pd.DataFrame(columns=avg_columns + min_columns + max_columns)], axis=1)

# Iterate over each dataset
for key, dataset in datasets.items():
    # Calculate the minimum and maximum values for each column starting from the third column
    deviations = dataset.iloc[2:, 2:].std()
    min_values = dataset.iloc[2:, 2:].min()
    max_values = dataset.iloc[2:, 2:].max()
    
    # Assign the minimum values to the corresponding row in riassunto
    
    row_index = dataframe_names.index(key)

    for col_index, deviations in enumerate(deviations, start=2):
        riassunto.iloc[row_index, len(riassunto.columns)//3 + col_index] = deviations
    
    for col_index, min_value in enumerate(min_values, start=2):
        riassunto.iloc[row_index, len(riassunto.columns)//3 + col_index] = min_value
    
    # Assign the maximum values to the corresponding row in riassunto
    for col_index, max_value in enumerate(max_values, start=2):
        riassunto.iloc[row_index, 2*len(riassunto.columns)//3 + col_index] = max_value

riassunto.head()



c:\Users\e2023898\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\e2023898\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\e2023898\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Note,Time Stamp,RP1,RP6,RP7,RV3V21,RV3V24,RV3V26,RFANS,RVP_A,...,NR_CONNECT_max,RT-SPARE2_max,RT-SPARE3_max,RT-SPARE4_max,RT-SPARE5_max,RT-SPARE6_max,RT-SPARE7_max,RT-SPARE8_max,RT-SPARE9_max,RT-SPARE10_max
0,R134A precarica 500 gr VT tutta aperta,NaN,0.0,0.0,0.0,15.062857,10.565714,7.371429,40.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,aggiunto 100 gr tot 600 gr VT tutta aperta,NaN,0.0,0.0,0.0,18.336,11.7,6.270667,40.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,aggiunto 100 gr tot 700 gr VT tutta aperta,NaN,0.0,0.0,0.0,18.461538,12.5,1.461538,40.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,aggiunto 100 gr tot 800 gr VT tutta aperta,NaN,0.0,0.0,0.0,20.288732,12.8,1.046479,40.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
    # Prepare test_data from test_id
    test_data = test_id.transpose()
    test_data.columns = test_data.iloc[0]
    test_data = test_data[1:]
test_data.head()

,STAZIONE DI COLLAUDO,OPERATORE,CODICE MACCHINA,MATRICOLA MACCHINA,NUMERO ODL,DATA/ORA COLLAUDO,FLUIDO,GAS REFRIGERANTE,REVISIONE,LINEA DEL FLUIDO,NR CIRCUITI,RESPONSABILE LINEA CHILLER\CLIMA,DIREZIONE TECNICA,ALIMENTAZIONE,NaN,ESITO COLLAUDO,NOTE DI COLLAUDO
1,R110,1 RESTANI MATTEO,NaN,NaN,NaN,27/02/2024 12:13:40 - 28/02/2024 08:13:07,GLICOLE,R134a_PA.txt,NaN,BASSA PORTATA,1,Ing. Paolo Russo,Ing. Paolo Russo,400V 3PH 50Hz,,PASSED,NaN


In [75]:
test_data_columns = test_data.columns.tolist()

# Step 2: Create z new columns in dataset_1 at position column 2 to column (2+z)
for i, col in enumerate(test_data_columns):
    dataset_1.insert(2 + i, col, test_data[col].values[0])

ValueError: cannot insert STAZIONE DI COLLAUDO, already exists

In [88]:
test_id = test_id.drop(14)

test_id.head(20)


,0,1
0,STAZIONE DI COLLAUDO,R110
1,OPERATORE,1 RESTANI MATTEO
2,CODICE MACCHINA,NaN
3,MATRICOLA MACCHINA,NaN
4,NUMERO ODL,NaN
5,DATA/ORA COLLAUDO,27/02/2024 12:13:40 - 28/02/2024 08:13:07
6,FLUIDO,GLICOLE
7,GAS REFRIGERANTE,R134a_PA.txt
8,REVISIONE,NaN
9,LINEA DEL FLUIDO,BASSA PORTATA


In [90]:
test_data = test_id.transpose()
test_data.columns = test_data.iloc[0]
test_data = test_data[1:]

In [ ]:
test_data_values = test_data.iloc[]

In [91]:
test_data.head()

,STAZIONE DI COLLAUDO,OPERATORE,CODICE MACCHINA,MATRICOLA MACCHINA,NUMERO ODL,DATA/ORA COLLAUDO,FLUIDO,GAS REFRIGERANTE,REVISIONE,LINEA DEL FLUIDO,NR CIRCUITI,RESPONSABILE LINEA CHILLER\CLIMA,DIREZIONE TECNICA,ALIMENTAZIONE,ESITO COLLAUDO,NOTE DI COLLAUDO
1,R110,1 RESTANI MATTEO,NaN,NaN,NaN,27/02/2024 12:13:40 - 28/02/2024 08:13:07,GLICOLE,R134a_PA.txt,NaN,BASSA PORTATA,1,Ing. Paolo Russo,Ing. Paolo Russo,400V 3PH 50Hz,PASSED,NaN
